<h1> Ensemble script for FATES-SP runs </h1>

This script will 

<ol>
<li>Clone github repo and move to CLM-FATES-SP pull request branchs
<li>Make a default 4x5 CLM-FATES-SP case
<li>Make an ensemble of CLM-FATES-SP cases 
<li>Generate an ensemble of parameter files with one at a time modification
<li>Point each ensemble member at a different parameter file
<li>Submit all these jobs
<li>Analyse output
</ol>

<h4>n.b. 1. Most of the decisions here on compsets, FATES options, resolution, variables etc. are easy to modify. </h4>
<h4>n.b. 2. This notebook is also set up to wok with Cheyenne specific paths. Making it machine agnostic is tbd.</h4>

<h3>Set global variables and paths for ensemble analysis </h3>

In [1]:
#set global variables to pass into BASH commands. 

ctsmrepo='ctsmsept20'
defbuildroot='/glade/scratch/'

#Settings for the SP ensemble
sp=1
noc4=1
if(sp == 1):
    resub=0
    ncases=14
    ychoose =2009
    defcase='SPdefault'
    defbuildroot='/glade/scratch/'
    if(noc4 == 1):
        ens_directory='SPfates_ens_noc4'
        caseroot='spfates_noc4_n'
        paramsdir='param_files_sp_noc4'
        paramfiledefault='fates_params_default_noc4.nc'
    else:
        ens_directory='SPfates_ens'
        caseroot='spfates_01_n'
        paramsdir='param_files_sp'
        paramfiledefault='fates_params_default.nc'
else:
#Settings for the NOCOMP ensemble. 
#set global variables to pass into BASH commands. 
    sp=0
    defcase='SPdefault'
    ens_directory='NOCOMP_fates_ens'
    caseroot='nocomp_fates_01_n'
    paramsdir='param_files_sp'
    paramfiledefault='fates_params_default.nc'    
    resub=2
    ncases=14
    ychoose =2009

<h3> Control variables for the analysis script.</h3>

In [2]:
dosubmit=1
forcenewcase=1
doanalysis=0

In [3]:
import os
notebookdr=os.getcwd() 

<h3>Clone CTSM repo, switch to SP pull request code.</h3>

In [4]:
%%bash -s "$ctsmrepo"
ctsmrepo=$1

cd /glade/work/$USER/
#go to git directory
if [[ -d "git" ]]
then
cd git/
else
mkdir git
cd git
fi

#go to git repo, or checkout code
if [[ -d "$ctsmrepo" ]]
then
cd $ctsmrepo
else

#clone CTSM code
git clone https://github.com/escomp/ctsm $ctsmrepo
cd $ctsmrepo
./manage_externals/checkout_externals
cd src

#change CTSM code to SP branch. 
git remote add  rosie_ctsm_repo  https://github.com/rosiealice/ctsm
git fetch rosie_ctsm_repo
git checkout -b SP_api rosie_ctsm_repo/SP_api

#change FATES code to SP branch. 
cd fates
git remote add  rosie_fates_repo https://github.com/rosiealice/fates
git fetch rosie_fates_repo
git checkout -b fates_sp_sept rosie_fates_repo/fates_sp_sept
fi

<h3> Make a default FATES 4x5 case </h3>

In [5]:
%%bash -s "$ctsmrepo" "$defcase"

ctsmrepo=$1
defcase=$2

cd /glade/work/$USER/git/$ctsmrepo/cime/scripts
pwd
if [[ -d "$defcase" ]]
then
    echo "$defcase exists on your filesystem."
else
./create_newcase --case $defcase --compset 2000_DATM%GSWP3v1_CLM50%FATES_SICE_SOCN_MOSART_CISM2%NOEVOLVE_SWAV_SIAC_SESP --res f45_f45_mg37  --run-unsupported
./case.setup
./case.build
fi

cd $casename

/glade/work/rfisher/git/ctsmsept20/cime/scripts
SPdefault exists on your filesystem.


<h3> Adjust settings of default case. </h3>

In [6]:
%%bash -s "$ctsmrepo" "$defcase" "$defbuildroot" "resub"

ctsmrepo=$1
defcase=$2
resub=$3
defbuildroot=$3
sp=$4
defbld=$defbuildroot$USER/$defcase/bld
echo $defbld
cd /glade/work/$USER/git/$ctsmrepo/cime/scripts/$defcase

echo 'updating settings',$newcase
./xmlchange BUILD_COMPLETE=TRUE;
./xmlchange EXEROOT=$defbld;
./xmlchange CONTINUE_RUN=FALSE;
./xmlchange --id STOP_N --val 10;
./xmlchange --id RESUBMIT --val $resub;
./xmlchange --id STOP_OPTION --val nyears;
./xmlchange --id CLM_FORCE_COLDSTART --val on;


/glade/scratch/rfisher/SPdefault/bld
updating settings,
For your changes to take effect, run:
./case.build --clean-all
./case.build
For your changes to take effect, run:
./case.build --clean-all
./case.build


ERROR: Entry RESUBMIT was listed as type int but value '/glade/scratch/' is not valid int


<h3> Make Default user_nl_clm File </h3>

In [7]:
%%bash -s "$ctsmrepo" "$defcase" "$notebookdr" "$paramsdir"

ctsmrepo=$1
echo $ctsmrepo
defcase=$2
notebookdr=$3
paramsdir=$4
echo $paramsdir
cd /glade/work/$USER/git/$ctsmrepo/cime/scripts
pwd
cp  $defcase/user_nl_clm $notebookdr/$paramsdir/user_nl_clm_default
echo $notebookdr/$paramsdir/
cd $notebookdr/$paramsdir/

pftfilename_def=$notebookdr/$paramsdir

echo use_fates_sp=.true. >> user_nl_clm_default
echo use_fates_nocomp=.true. >> user_nl_clm_default
echo use_fates_fixed_biogeog=.true. >> user_nl_clm_default
echo fates_paramfile=\'\pftfilename_default\' >> user_nl_clm_default

#cat  $notebookdr/$paramsdir/user_nl_clm_default

ctsmsept20
param_files_sp_noc4
/glade/work/rfisher/git/ctsmsept20/cime/scripts
/glade/u/home/rfisher/fates_sp_ensembles/param_files_sp_noc4/


<h3> Make ensemble of cases.</h3>

In [8]:
%%bash -s "$ctsmrepo" "$defcase" "$ens_directory" "$caseroot" $ncases "$forcenewcase"

ctsmrepo=$1
defcase=$2
ens_directory=$3
caseroot=$4
ncases=$5
forcenewcase=$6

echo $ncases
echo $caseroot
echo $ens_directory
echo $ctsmrepo

cd /glade/work/$USER/git/$ctsmrepo/cime/scripts
pwd

if [[ -d "$ens_directory" ]]
then
echo "directory already exists"
else
mkdir $ens_directory
fi 

counter1=0 #include a default zero case. 
while [ $counter1 -le $ncases ]
do
  newcase=$caseroot$counter1
  rm -rf $newcase 
  if [ -d $ens_directory/$newcase ]
  then
    echo ' new case already exists',$ens_directory/$newcase   
  else
   echo 'making', $newcase
    ./create_clone --clone $defcase --case $ens_directory/$newcase ;
    cd $ens_directory/$newcase 
    ./case.setup;
    cd ../../
  fi
  if [ $forcenewcase -eq 1 ]
  then
   echo 'force making', $ens_directory/$newcase
    rm -rf $ens_directory/$newcase
    ./create_clone --clone $defcase --case $ens_directory/$newcase ;
    cd $ens_directory/$newcase
    ./case.setup;
    cd ../../
  fi

  ((counter1++))
  done    


14
spfates_noc4_n
SPfates_ens_noc4
ctsmsept20
/glade/work/rfisher/git/ctsmsept20/cime/scripts
directory already exists
 new case already exists,SPfates_ens_noc4/spfates_noc4_n0
force making, SPfates_ens_noc4/spfates_noc4_n0
 Successfully created new case spfates_noc4_n0 from clone case SPdefault 
Setting resource.RLIMIT_STACK to -1 from (-1, -1)
job is case.run USER_REQUESTED_WALLTIME None USER_REQUESTED_QUEUE None WALLTIME_FORMAT %H:%M:%S
Creating batch scripts
Writing case.run script from input template /glade/work/rfisher/git/ctsmsept20/cime/config/cesm/machines/template.case.run
Creating file .case.run
Writing case.st_archive script from input template /glade/work/rfisher/git/ctsmsept20/cime/config/cesm/machines/template.st_archive
Creating file case.st_archive
If an old case build already exists, might want to run 'case.build --clean' before building
You can now run './preview_run' to get more info on how your case will be run
Setting resource.RLIMIT_STACK to -1 from (-1, -1)
Mach

/glade/work/rfisher/git/ctsmsept20/cime/scripts/SPfates_ens_noc4/spfates_noc4_n0/env_mach_specific.xml already exists, delete to replace
/glade/work/rfisher/git/ctsmsept20/cime/scripts/SPfates_ens_noc4/spfates_noc4_n0/env_mach_specific.xml already exists, delete to replace
/glade/work/rfisher/git/ctsmsept20/cime/scripts/SPfates_ens_noc4/spfates_noc4_n1/env_mach_specific.xml already exists, delete to replace
/glade/work/rfisher/git/ctsmsept20/cime/scripts/SPfates_ens_noc4/spfates_noc4_n1/env_mach_specific.xml already exists, delete to replace
/glade/work/rfisher/git/ctsmsept20/cime/scripts/SPfates_ens_noc4/spfates_noc4_n2/env_mach_specific.xml already exists, delete to replace
/glade/work/rfisher/git/ctsmsept20/cime/scripts/SPfates_ens_noc4/spfates_noc4_n2/env_mach_specific.xml already exists, delete to replace
/glade/work/rfisher/git/ctsmsept20/cime/scripts/SPfates_ens_noc4/spfates_noc4_n3/env_mach_specific.xml already exists, delete to replace
/glade/work/rfisher/git/ctsmsept20/cime/s

<h3> Create .nc version of current defualt file. I FAILED ATT THIS. NEEDS INPUT..</h3>
You currently have to do this step manually, by doing this in the terminal:

<i> cd /glade/work/$USER/git/$ctsmrepo/src/fates/parameter_files/</i><br>
<i>ncgen -o fates_params_default.nc fates_params_default.cdl </i>

<h3> Make a directory for the parameter files to go in and copy default .nc parameter file there. </h3>

In [15]:
%%bash -s "$ctsmrepo" "$notebookdr"  "$paramsdir" "$paramfiledefault"
echo $1
ctsmrepo=$1
notebookdr=$2
paramsdir=$3
paramfiledefault=$4
echo $ctsmrepo


cd $notebookdr
if [[ -d "$paramsdir" ]]
then
   cd $paramsdir
   echo "existing param firectory"
else 
    mkdir $paramsdir
    cd $paramsdir
    echo "made param firectory"
fi
pwd
echo $paramfiledefault
cp /glade/work/$USER/git/$ctsmrepo/src/fates/parameter_files/$paramfiledefault $paramfiledefault



ctsmsept20
ctsmsept20
existing param firectory
/glade/u/home/rfisher/fates_sp_ensembles/param_files_sp_noc4
fates_params_default_noc4.nc


<h3>Mpdify default parameter file</h3>

In [16]:
import netCDF4 as nc4
import sys
import os
import shutil

def copy_clobber(filename1, filename2):
    try:
        os.remove(filename2)
       # print('replacing file: '+filename2)
    except:
        print('file does not yet exist: '+filename2)
    shutil.copyfile(filename1, filename2)


filename_template = notebookdr+'/'+paramsdir+'/'+paramfiledefault

def def_pftfile(notebookdr, paramsdir,i,variable,delta):
    filename_out= notebookdr+'/'+paramsdir+'/fates_params_update_'+str(i)+'.nc'
    print(filename_out)
    print(filename_template)
    copy_clobber(filename_template,filename_out)

    fin = nc4.Dataset(filename_out, 'r+')
    print(filename_out)
    var = fin.variables[variable]
    var[:] = var[:]*delta
    fin.close()
    
#1 fates_leaf_clumping_index +-10% (not above 1)
#2 fates_leaf_diameter +- 20%
#3 fates_leaf_stomatal_slope_ballberry +- 20%
#4 fates_leaf_vcmax25top +- 20%
#5 Fates_leaf_xl = +- 20%
#6 fates_rholvis=+-20-%
#7 Fates_smpsc=+-20%
#default    
print(paramsdir)
def_pftfile(notebookdr, paramsdir,0,'fates_leaf_clumping_index',1.0)

    
def_pftfile(notebookdr, paramsdir,1,'fates_leaf_clumping_index',0.9)
def_pftfile(notebookdr, paramsdir,2,'fates_leaf_clumping_index',1.1)

def_pftfile(notebookdr, paramsdir,3,'fates_leaf_diameter',0.8)
def_pftfile(notebookdr, paramsdir,4,'fates_leaf_diameter',1.2)

def_pftfile(notebookdr, paramsdir,5,'fates_leaf_stomatal_slope_ballberry',0.8)
def_pftfile(notebookdr, paramsdir,6,'fates_leaf_stomatal_slope_ballberry',1.2)

def_pftfile(notebookdr, paramsdir,7,'fates_leaf_vcmax25top',0.8)
def_pftfile(notebookdr, paramsdir,8,'fates_leaf_vcmax25top',1.2)

def_pftfile(notebookdr, paramsdir,9,'fates_leaf_xl',0.8)
def_pftfile(notebookdr, paramsdir,10,'fates_leaf_xl',1.2)

def_pftfile(notebookdr, paramsdir,11,'fates_rholvis',0.8)
def_pftfile(notebookdr, paramsdir,12,'fates_rholvis',1.2)

def_pftfile(notebookdr, paramsdir,13,'fates_smpsc',0.8)
def_pftfile(notebookdr, paramsdir,14,'fates_smpsc',1.2)



param_files_sp_noc4
/glade/u/home/rfisher/fates_sp_ensembles/param_files_sp_noc4/fates_params_update_0.nc
/glade/u/home/rfisher/fates_sp_ensembles/param_files_sp_noc4/fates_params_default_noc4.nc
file does not yet exist: /glade/u/home/rfisher/fates_sp_ensembles/param_files_sp_noc4/fates_params_update_0.nc
/glade/u/home/rfisher/fates_sp_ensembles/param_files_sp_noc4/fates_params_update_0.nc
/glade/u/home/rfisher/fates_sp_ensembles/param_files_sp_noc4/fates_params_update_1.nc
/glade/u/home/rfisher/fates_sp_ensembles/param_files_sp_noc4/fates_params_default_noc4.nc
file does not yet exist: /glade/u/home/rfisher/fates_sp_ensembles/param_files_sp_noc4/fates_params_update_1.nc
/glade/u/home/rfisher/fates_sp_ensembles/param_files_sp_noc4/fates_params_update_1.nc
/glade/u/home/rfisher/fates_sp_ensembles/param_files_sp_noc4/fates_params_update_2.nc
/glade/u/home/rfisher/fates_sp_ensembles/param_files_sp_noc4/fates_params_default_noc4.nc
file does not yet exist: /glade/u/home/rfisher/fates_sp_en

<h3> Point each ensemble script at different parameter file </h3>

In [17]:
%%bash -s "$ctsmrepo" "$defcase" "$ens_directory" "$caseroot" $ncases "$notebookdr"  "$paramsdir"

ctsmrepo=$1
defcase=$2
ens_directory=$3
caseroot=$4
ncases=$5
forcenewcase=1
notebookdir=$6
paramsdir=$7

unl=user_nl_clm
cd /glade/work/$USER/git/$ctsmrepo/cime/scripts
counter1=0
cd $ens_directory
#cat $notebookdir/$paramsdir/user_nl_clm_default
while [ $counter1 -le $ncases ]
do
  newcase=$caseroot$counter1
  pftfilename=$notebookdir/$paramsdir/fates_params_update_$counter1.nc
  if [ -d $newcase ]
  then
   cd $newcase
   #get default parameter file
    cp $notebookdir/$paramsdir/user_nl_clm_default user_nl_clm
    sed -i "s|pftfilename_default|$pftfilename|g" user_nl_clm
    cd ../
  else
   echo 'no case', $newcase
  fi


  ((counter1++))
  done
cat $newcase/user_nl_clm

!----------------------------------------------------------------------------------
! Users should add all user specific namelist changes below in the form of 
! namelist_var = new_namelist_value 
!
! EXCEPTIONS: 
! Set use_cndv           by the compset you use and the CLM_BLDNML_OPTS -dynamic_vegetation setting
! Set use_vichydro       by the compset you use and the CLM_BLDNML_OPTS -vichydro           setting
! Set use_cn             by the compset you use and CLM_BLDNML_OPTS -bgc  setting
! Set use_crop           by the compset you use and CLM_BLDNML_OPTS -crop setting
! Set spinup_state       by the CLM_BLDNML_OPTS -bgc_spinup      setting
! Set co2_ppmv           with CCSM_CO2_PPMV                      option
! Set dtime              with L_NCPL                             option
! Set fatmlndfrc         with LND_DOMAIN_PATH/LND_DOMAIN_FILE    options
! Set finidat            with RUN_REFCASE/RUN_REFDATE/RUN_REFTOD options for hybrid or branch cases
!                        (includ

<h3>Submit ensemble of cases</h3>

In [18]:
%%bash -s "$ctsmrepo" "$defcase" "$ens_directory" "$caseroot" $ncases "$dosubmit" 

ctsmrepo=$1
defcase=$2
ens_directory=$3
caseroot=$4
ncases=$5
dosubmit=$6


if [ $dosubmit -eq 1 ]
then
cd /glade/work/$USER/git/$ctsmrepo/cime/scripts

counter1=0
cd $ens_directory
pwd
ls -l
while [ $counter1 -le $ncases ]
do
  echo $counter1
  newcase=$caseroot$counter1
  echo $newcase

  if [ -d $newcase ]
  then
    cd $newcase
    echo 'submitting job',$newcase
    ./xmlchange BUILD_COMPLETE=TRUE
    ./case.submit;
    cd ../
  else
   echo 'no case', $newcase
  fi

  ((counter1++))
  done
else
echo "not submitting jobs"
fi

/glade/work/rfisher/git/ctsmsept20/cime/scripts/SPfates_ens_noc4
total 15
drwxr-xr-x 7 rfisher ncar 4096 Nov 18 08:36 spfates_noc4_n0
drwxr-xr-x 7 rfisher ncar 4096 Nov 18 08:36 spfates_noc4_n1
drwxr-xr-x 7 rfisher ncar 4096 Nov 18 08:36 spfates_noc4_n10
drwxr-xr-x 7 rfisher ncar 4096 Nov 18 08:36 spfates_noc4_n11
drwxr-xr-x 7 rfisher ncar 4096 Nov 18 08:36 spfates_noc4_n12
drwxr-xr-x 7 rfisher ncar 4096 Nov 18 08:36 spfates_noc4_n13
drwxr-xr-x 7 rfisher ncar 4096 Nov 18 08:36 spfates_noc4_n14
drwxr-xr-x 7 rfisher ncar 4096 Nov 18 08:36 spfates_noc4_n2
drwxr-xr-x 7 rfisher ncar 4096 Nov 18 08:36 spfates_noc4_n3
drwxr-xr-x 7 rfisher ncar 4096 Nov 18 08:36 spfates_noc4_n4
drwxr-xr-x 7 rfisher ncar 4096 Nov 18 08:36 spfates_noc4_n5
drwxr-xr-x 7 rfisher ncar 4096 Nov 18 08:36 spfates_noc4_n6
drwxr-xr-x 7 rfisher ncar 4096 Nov 18 08:36 spfates_noc4_n7
drwxr-xr-x 7 rfisher ncar 4096 Nov 18 08:36 spfates_noc4_n8
drwxr-xr-x 7 rfisher ncar 4096 Nov 18 08:36 spfates_noc4_n9
0
spfates_noc4_n0
sub


env_batch.xml appears to have changed, regenerating batch scripts
manual edits to these file will be lost!

Model clm no file specified for finidat
submit_jobs case.run
Submit job case.run
Submit job case.st_archive

env_batch.xml appears to have changed, regenerating batch scripts
manual edits to these file will be lost!

Model clm no file specified for finidat
submit_jobs case.run
Submit job case.run
Submit job case.st_archive

env_batch.xml appears to have changed, regenerating batch scripts
manual edits to these file will be lost!

Model clm no file specified for finidat
submit_jobs case.run
Submit job case.run
Submit job case.st_archive

env_batch.xml appears to have changed, regenerating batch scripts
manual edits to these file will be lost!

Model clm no file specified for finidat
submit_jobs case.run
Submit job case.run
Submit job case.st_archive

env_batch.xml appears to have changed, regenerating batch scripts
manual edits to these file will be lost!

Model clm no file speci

<h2>Analyse output</h2>

In [ ]:
%matplotlib inline
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import xarray as xr
from matplotlib import pyplot as plt
import datetime
import cartopy
import cartopy.crs as ccrs
import os.path
import xesmf as xe

In [ ]:

titles=['clumping_index -','clumping_index +','leaf_diameter -','leaf_diameter +',
           'stomatal_slope -','stomatal_slope +','vcmax25top -','vcmax25top +',
           'leaf_xl -','leaf_xl +','rholvis -','rholvis +',    
           'smpsc -','smpsc +' ]
output='/glade/scratch/rfisher/'
conv = 3600*24*365
yr='.clm2.h0.'

<h3> Get ILAMB data</h3>

In [ ]:
if doanalysis ==1: 
    filenameb ='/glade/u/home/rfisher/GBAF_Benchmark.nc'
    gppbm = xr.open_dataset(filenameb,group='MeanState')

    filenameb ='/glade/u/home/rfisher/MODIS_Benchmark.nc'
    etbm = xr.open_dataset(filenameb,group='MeanState')

    filenameef ='/glade/u/home/rfisher/GBAF_Benchmark_EF.nc'
    efbm = xr.open_dataset(filenameef,group='MeanState')


<h3> Regrid ILAMB data</h3>

In [ ]:
ds=gppbm.timeint_of_gpp
ds_out = xr.Dataset({'lat': (['lat'], np.arange(-90, 90, 4.0)),
                     'lon': (['lon'], np.arange(-180, 180, 5.0)),
                    }
                   )
regridder = xe.Regridder(ds, ds_out, 'bilinear')
regridder.clean_weight_file()
regridder
ds_out_obs = regridder(ds)
ds_out_obs = ds_out_obs.where(ds_out_obs<10000,0,0)

ds=etbm.timeint_of_et
regridder = xe.Regridder(ds, ds_out, 'bilinear')
regridder.clean_weight_file()
regridder
ds_out_ET_obs = regridder(ds)
ds_out_ET_obs = ds_out_obs.where(ds_out_ET_obs<10000,0,0)
#ds_out_ET_obs.plot()

ds=efbm.timeint_of_EvapFrac
regridder = xe.Regridder(ds, ds_out, 'bilinear')
regridder.clean_weight_file()
regridder
ds_out_EF_obs = regridder(ds)
ds_out_EF_obs = ds_out_EF_obs.where(ds_out_EF_obs<10000,0,0)
ds_out_EF_obs.plot(vmax=1,cmap='jet')



In [ ]:
efbm

<h4>Default ensemble member</h4>

In [ ]:

run=caseroot+str(0)
arc = output + 'archive/' + run + '/lnd/hist/'
# arc = output + run + '/run/'   
f2=(arc +run+yr+str(ychoose)+'-01.nc') 
print(f2)
if(os.path.isfile(f2)):
    dsdef = xr.open_mfdataset(arc +run+yr+str(ychoose)+'*', decode_times=False)
else:
    arc = output + run + '/run/'
    dsdef = xr.open_mfdataset(arc +run+yr+str(ychoose)+'*', decode_times=False)
print(arc)

<h3>Make output figure directory</h3>

In [ ]:
%%bash -s "$notebookdr"  
notebookdr=$2

cd $notebookdr
if [[ -d "figs" ]]
then
   echo "existing fig firectory"
else 
    mkdir figs
    echo "made fig firectory"
fi

<h3>Read in selected year of output data</h3>

<h3>GPP</h3>

In [ ]:
import time
delta=0
if doanalysis ==1:
    conv=24*3600  
    fig = plt.figure()
    plt.rcParams['figure.figsize'] = [16,20]
    fig.subplots_adjust(hspace=0.1, wspace=0.1)
    vs=[1, 2, 5,6, 7, 8]
    print(vs)
    count=1
    for i in vs:
        print(i)
        ax = fig.add_subplot(3, 2, count)
        count=count+1
        run=caseroot+str(i)
        arc = output + 'archive/' + run + '/lnd/hist/'
       # arc = output + run + '/run/'
        f2=(arc +run+yr+str(ychoose)+'-01.nc') 
        if(os.path.isfile(f2)):
            ds0 = xr.open_mfdataset(arc +run+yr+str(ychoose)+'*', decode_times=False)
        else:
            arc = output + run + '/run/'
            ds0 = xr.open_mfdataset(arc +run+yr+str(ychoose)+'*', decode_times=False)
        print(arc) 
        
        if(delta == 1):
             delt=np.multiply(np.mean(ds0.GPP,0),conv)-np.multiply(np.mean(dsdef.GPP,0),conv)
             plt1=delt.plot(vmin=-1, vmax=1,cmap='RdYlBu') 
        else:
             ds=np.multiply(np.mean(ds0.GPP,0),conv)
             ds_out_model = xr.Dataset({'lat': (['lat'], np.arange(-90, 90, 4.0)),
                     'lon': (['lon'], np.arange(-180, 180, 5)),})
             regridder = xe.Regridder(ds, ds_out_model, 'bilinear')
             regridder.clean_weight_file()
             regridder
             ds_out_model = regridder(ds)
             diff=ds_out_model-ds_out_obs
             plt1=diff.plot(vmin=-5, vmax=5,cmap='RdYlBu') 
        
        ax.set_title(titles[i-1]) 
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        
        fig.canvas.draw()
        time.sleep(1)  

if(delta == 1):
    fnmfig='figs/ensemble_GPP_delta_'+caseroot+'.png'
else:
    fnmfig='figs/ensemble_GPP__error_'+caseroot+'.png'
print(fnmfig)
plt.savefig(fnmfig)


<h3>ET</h3>

In [ ]:
import time
delta=0
if doanalysis ==1:
    conv=24*3600
    fig = plt.figure()
    plt.rcParams['figure.figsize'] = [16,20]
    fig.subplots_adjust(hspace=0.1, wspace=0.1)
    vs=[1, 2, 5,6, 7, 8]
    print(vs)
    count=1
    for i in vs:
        print(i)
        ax = fig.add_subplot(3, 2, count)
        count=count+1
        run=caseroot+str(i)
        arc = output + 'archive/' + run + '/lnd/hist/'
       # arc = output + run + '/run/'
        f2=(arc +run+yr+str(ychoose)+'-01.nc') 
        if(os.path.isfile(f2)):
            ds0 = xr.open_mfdataset(arc +run+yr+str(ychoose)+'*', decode_times=False)
        else:
            arc = output + run + '/run/'
            ds0 = xr.open_mfdataset(arc +run+yr+str(ychoose)+'*', decode_times=False)
        print(arc) 
        
        if(delta == 1):
             delt=np.multiply(np.mean(ds0.QFLX_EVAP_TOT,0),conv)-np.multiply(np.mean(dsdef.QFLX_EVAP_TOT,0),conv)
             plt1=delt.plot(vmin=-0.2, vmax=0.2,cmap='RdYlBu') 
        else:
             ds=np.multiply(np.mean(ds0.QFLX_EVAP_TOT,0),conv)
             ds_out = xr.Dataset({'lat': (['lat'], np.arange(-90, 90, 4.0)),
                     'lon': (['lon'], np.arange(-180, 180, 5)),})
             regridder = xe.Regridder(ds, ds_out_model, 'bilinear')
             regridder.clean_weight_file()
             regridder
             ds_out_model = regridder(ds)
             diff=ds_out_model-ds_out_ET_obs
             plt1=diff.plot(vmin=-6, vmax=6,cmap='RdYlBu') 
        

        #plt1=ds_out_model.plot(vmin=0, vmax=12,cmap='bwr') 
        ax.set_title(titles[i-1]) 
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        
        fig.canvas.draw()
        time.sleep(1)  

if(delta == 1):
    fnmfig='figs/ensemble_ET_delta_'+caseroot+'.png'
else:
    fnmfig='figs/ensemble_ET__error_'+caseroot+'.png'
print(fnmfig)
plt.savefig(fnmfig)


<h3> Evap Frac </h3>

In [ ]:
import time
delta=0
if doanalysis ==1:
    conv=24*3600
    fig = plt.figure()
    plt.rcParams['figure.figsize'] = [16,20]
    fig.subplots_adjust(hspace=0.1, wspace=0.1)
    vs=[1, 2, 5,6, 7, 8]
    print(vs)
    count=1
    for i in vs:
        print(i)
        ax = fig.add_subplot(3, 2, count)
        count=count+1
        run=caseroot+str(i)
        arc = output + 'archive/' + run + '/lnd/hist/'
       # arc = output + run + '/run/'
        f2=(arc +run+yr+str(ychoose)+'-01.nc') 
        if(os.path.isfile(f2)):
            ds0 = xr.open_mfdataset(arc +run+yr+str(ychoose)+'*', decode_times=False)
        else:
            arc = output + run + '/run/'
            ds0 = xr.open_mfdataset(arc +run+yr+str(ychoose)+'*', decode_times=False)
        print(arc) 
        
        if(delta == 1):
             delt=np.multiply(np.mean(ds0.QFLX_EVAP_TOT,0),conv)-np.multiply(np.mean(dsdef.QFLX_EVAP_TOT,0),conv)
             plt1=delt.plot(vmin=-0.2, vmax=0.2,cmap='RdYlBu') 
        else:
             ds=np.multiply(np.mean(ds0.QFLX_EVAP_TOT,0),conv)
             ds_out = xr.Dataset({'lat': (['lat'], np.arange(-90, 90, 4.0)),
                     'lon': (['lon'], np.arange(-180, 180, 5)),})
             regridder = xe.Regridder(ds, ds_out_model, 'bilinear')
             regridder.clean_weight_file()
             regridder
             ds_out_model = regridder(ds)
             diff=ds_out_model-ds_out_ET_obs
             plt1=diff.plot(vmin=-6, vmax=6,cmap='RdYlBu') 
        

        #plt1=ds_out_model.plot(vmin=0, vmax=12,cmap='bwr') 
        ax.set_title(titles[i-1]) 
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        
        fig.canvas.draw()
        time.sleep(1)  

if(delta == 1):
    fnmfig='figs/ensemble_ET_delta_'+caseroot+'.png'
else:
    fnmfig='figs/ensemble_ET__error_'+caseroot+'.png'
print(fnmfig)
plt.savefig(fnmfig)
